<a href="https://colab.research.google.com/github/AshutoshPdy22/FUTURE_ML_01/blob/main/task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install prophet

In [ ]:

# Step 2: Import libraries
import pandas as pd
import numpy as np
import os
from prophet import Prophet
import matplotlib.pyplot as plt
import holidays

# Step 3: Load and clean data
df = pd.read_csv("retail_sales.csv")

# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Ensure Total Amount is numeric
df['Total Amount'] = pd.to_numeric(df['Total Amount'], errors='coerce')

# Step 4: Aggregate daily sales
daily_sales = df.groupby('Date', as_index=False)['Total Amount'].sum()
daily_sales = daily_sales.rename(columns={'Date': 'ds', 'Total Amount': 'y'})

# Step 5: Feature Engineering

# Add month and weekday
daily_sales['month'] = daily_sales['ds'].dt.month
daily_sales['day_of_week'] = daily_sales['ds'].dt.dayofweek

# Add "holiday" flag using national holidays
us_holidays = holidays.US()
daily_sales['is_holiday'] = daily_sales['ds'].apply(lambda x: 1 if x in us_holidays else 0)

# Add rolling averages (e.g., 7-day and monthly avg)
daily_sales['y_7d_avg'] = daily_sales['y'].rolling(window=7).mean()
daily_sales['y_30d_avg'] = daily_sales['y'].rolling(window=30).mean()

# Optional: Drop NaNs from rolling averages if modeling later
daily_sales.fillna(method='bfill', inplace=True)

# Step 6: Forecasting using Prophet
model_data = daily_sales[['ds', 'y']]

model = Prophet(holidays=pd.DataFrame({
    'ds': pd.to_datetime(list(us_holidays.keys())),
    'holiday': list(us_holidays.values())
}))
model.fit(model_data)

# Step 7: Make future predictions
future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)

# Step 8: Save forecast output
output = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
output = output.merge(daily_sales[['ds', 'y']], on='ds', how='left')

# Step 9: Export to CSV for Power BI
output.to_csv("sales_forecast_with_actuals.csv", index=False)
daily_sales.to_csv("cleaned_sales_data.csv", index=False)

# Step 10: Plot forecast and save image
fig = model.plot(forecast)
plt.title("Retail Sales Forecast (Prophet)")
plt.savefig("forecast_plot.png")
plt.close()

print("✅ Data cleaned, forecasted, and exported for Power BI.")


/tmp/ipython-input-397696302.py:37: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_sales.fillna(method='bfill', inplace=True)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjbi35j_b/3u0xnh_e.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjbi35j_b/ahbpcidk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=45419', 'data', 'file=/tmp/tmpjbi35j_b/3u0xnh_e.json', 'init=/tmp/tmpjbi35j_b/ahbpcidk.json', 'output', 'file=/tmp/tmpjbi35j_b/prophet_model5_oph9wu/prophet_model-20250806174355.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:43:55 - cm

✅ Data cleaned, forecasted, and exported for Power BI.
